In [ ]:
#Use this in Julia REPL: 
# using IJulia
#  IJulia.installkernel("Julia 12 Threads", env=Dict("JULIA_NUM_THREADS" => "12",))

using  CSV, DataFrames, Random, LinearAlgebra, Distributions, EcologicalNetworks

Lx=1000 
Ly=500
abun=1000
timepoints=10000
nbirth=ndeath=convert(Int64,floor(abun*0.1))

reps=1:2

disp=(2,5,10,15,30)
methodrefs=DataFrame(ind=[1, 2, 3], methods=["neutral","positive","negative"])
pars=collect(Iterators.product(disp,methodrefs[:,:ind]))

pars=pars[[1,2]]

@threads for i1 in reps

cd("C:\\Users\\mihir\\Documents\\landscapes")

landscape=CSV.read(string("landscape_",i1,".csv"), DataFrame)
      
ind=landscape[1,2]
rangepar=landscape[1,3]    
    
landscape[:,:pres]=shuffle([ones(Int8,abun);zeros(Int8,(nrow(landscape)-abun))])
#landscape[:,:rep]=fill(1,nrow(landscape))
#landscape[:,:rangepars]=fill(5,nrow(landscape))
    
initial=filter(:pres => x-> x .== 1,landscape)
initial=initial[:,[:rep,:rangepars,:x,:y,:pres]]

CSV.write(string("initial_",i1,".csv"),initial)

popmat=unstack(landscape[:,["y","x","pres"]],:y,:x,:pres)
popmat=Matrix(popmat[:,Not("y")])


habmat=unstack(landscape[:,["y","x","soiltype"]],:y,:x,:soiltype)
habmat=Matrix(habmat[:,Not("y")])

#Threshold for a suitable habitat
mid=quantile(landscape[:,:soiltype],0.5)


for i2 in 1:length(pars)
        
        disps=pars[i2][1]
        methods=pars[i2][2]
        
        #Define dispersal probability as a function of #neighbors for all three dispersal methods
        maxnb=((2*disps+1)^2)-1
        neigh=log.(maxnb,1:maxnb)
        disp_list=([0;fill(1,length(neigh))],[0;0.5 .+ (neigh ./2)],[0;1.0 .- neigh])  

        for i in 1:10

            samp=shuffle((findall(popmat.==1)))
            popmat[samp[1:ndeath]].=0

            j=0

            while j<nbirth

                x=sample(1:Ly)
                y=sample(1:Lx)

                if(popmat[x,y]==0)

                    xs=((x-disps):(x+disps))
                    xs=xs[0 .<xs .<=Ly]

                    ys=(y-disps):(y+disps)
                    ys=ys[0 .<ys .<=Lx]

                    nb=sum(popmat[xs,ys])+1

                    val=ceil(Int,habmat[x,y]-mid)*
                        ceil(Int,(disp_list[methods][nb]-rand()))

                    popmat[x,y]=val

                    j=j+val

                end
            end   

        end 

            popmat1=DataFrame(popmat,:auto)
            rename!(x-> strip(string(x), ['x']),popmat1)
            popmat1[:,:y].=1:500
            popmat1=stack(popmat1,1:1000)
            rename!(popmat1,:variable=> :x, :value => :pres)
            popmat1[!,:x] = parse.(Int64,popmat1[!,:x])
            filter!(:pres => x-> x .== 1,popmat1)
            popmat1[:,:rep].=ind
            popmat1[:,:rangepar].=rangepars
            popmat1[:,:disp].=disps
            popmat1[:,:method].=methodrefs[methods,2]
            
            CSV.write(string("popmat_",i1,"_",i2,".csv"),popmat1)

end
end  

In [1]:
#Functions to calculate clusters and further process the simulated data

using CSV, DataFrames, Random, LinearAlgebra, Distributions, Distances, 
CategoricalArrays, GLM, Plots, Optim


function divisors(Lx::Int64,Ly::Int64)
    
    d=collect(1:min(Lx,Ly))
    d=d[mod.(min(Lx,Ly),d) .==0]
    
    return sort(union(d, collect(2:2:ceil(Int64,min(Lx,Ly)/4))))
end
    

function perc_up(dat::DataFrame,Lx::Int64,Ly::Int64)
    
    d=divisors(Lx,Ly)
    
    a=DataFrame(vec(collect(Iterators.product(1:Lx,1:Ly))))
    
    select!(a,"1"=>"x","2"=>"y")
    
    dat=outerjoin(dat,a, on= [:x=>:x,:y=>:y],order=:right)
    
    replace!(dat.pres,missing=>0)
    
    result=DataFrame()
    
    for i in d
        
        cutx=ceil.(Int64,collect((1:Lx) ./ i))
        cuty=ceil.(Int64,collect((1:Ly) ./ i))
        
        dat1=deepcopy(dat)
        
        dat1[!,:x1] =cutx[dat1.x]
        dat1[!,:y1] = cuty[dat1.y]
        
        dat1=combine(groupby(dat1,[:x1,:y1]),:pres=>sum)
        
        mat1=unstack(dat1,:y1,:x1,:pres_sum)
        
        mat1=Matrix{Int64}(select(mat1,Not(:y1)))
        
        clust=zeros(Int64,size(mat1)[1],size(mat1)[2])
        
        largest=0
        
        lbl=collect(0:length(clust))
        
        rw=size(mat1)[1]
        cl=size(mat1)[2]
        
        for i2 in 1:cl 
            for i1 in 1:rw
                
                if mat1[i1,i2] > 0
                    
                    left= (i2-1)==0 ? 0 : clust[i1,i2-1]
                    above= (i1-1)==0 ? 0 : clust[i1-1,i2]
                    
                    if (left==0) && (above==0)
                        
                        largest+=1
                        clust[i1,i2]=largest
                        lbl[(rw*(i2-1))+i1]=largest
                        
                    else
                        
                        if (left!=0) && (above==0)
                            
                            clust[i1,i2]=clust[i1,i2-1]
                            lbl[(rw*(i2-1))+i1]=lbl[(rw*(i2-2))+i1]
                            
                        else
                            
                            if (left==0) && (above!=0)
                                
                                clust[i1,i2]=clust[i1-1,i2]
                                lbl[(rw*(i2-1))+i1]=lbl[(rw*(i2-1))+i1-1]
                                
                            else
                                
                                newlab=min(left,above)
                                lbl[findall(x-> x==left || x==above, lbl)] .=newlab
                                clust[i1,i2]=newlab
                                clust[findall(x-> x==max(left,above), clust)].=newlab
                            end
                        end
                    end
                end
            end
        end
        
        res=DataFrame(pres=reshape(mat1,length(mat1)),cluster=reshape(clust,length(clust)))
        res=combine(groupby(res,:cluster),:pres=>sum)
        res=res[res.cluster .!=0,:]

        freq=combine(groupby(res,:pres_sum),:pres_sum=>length)

        append!(result,DataFrame(d_cutoff=i,cluster=freq.pres_sum,freq=freq.pres_sum_length)) 
        
    end
    
    return result
end

function fish(a,b)
    
    a=log2.(a)
    b=log2.(b.+1)
    
    return coef(lm(@formula(a~b),DataFrame(a=a,b=b)))[2]
    
end
    

function weibull_fit(a,b)
        
        res=0.0
        
        try 
     
            res=shape(fit_mle(Weibull,wsample(a,b,100) .+ 1))
                
        catch err
            
            if isa(error, DomainError)
                
                res=0.0
                
            end
        end
        
        return res
        
end


function segmented_f(dat::DataFrame,d::Vector{Int64})
    
    min_dev = Inf
    best_bp = 0
    current1=current2=nothing
    
    for bp in 3:(length(d)-2)
        
        current1=lm(@formula(f~d_cutoff),dat[1:bp,:])
        current2=lm(@formula(f~d_cutoff),dat[(bp+1):length(d),:])
        
        if (deviance(current1)+deviance(current2)) < min_dev
            
            min_dev= deviance(current1)+deviance(current2)
            best_bp=bp
        end
    end
    
    return dat.d_cutoff[best_bp],min_dev
    
end

function segmented_cm(dat::DataFrame,d::Vector{Int64})
    
    min_dev = Inf
    best_bp = 0
    current1=current2=nothing
    
    for bp in 3:(length(d)-2)
        
        current1=lm(@formula(cm~d_cutoff),dat[1:bp,:])
        current2=lm(@formula(cm~d_cutoff),dat[(bp+1):length(d),:])
        
        if (deviance(current1)+deviance(current2)) < min_dev
            
            min_dev= deviance(current1)+deviance(current2)
            best_bp=bp
        end
    end
    
    return dat.d_cutoff[best_bp],min_dev
    
end
    
    
function get_f_cm(dat,Lx,Ly)
    
    res=perc_up(dat,Lx,Ly)

    a=DataFrame(vec(collect(Iterators.product(divisors(Lx,Ly),1:sum(dat.pres)))))

    select!(a,"1"=>"d_cutoff","2"=>"cluster")

    res1=outerjoin(res,a, on= [:d_cutoff=>:d_cutoff,:cluster=>:cluster],order=:right)

    replace!(res1.freq,missing=>0)
    
    res1.freq =identity.(res1.freq)

    fishdat=DataFrame()
    wshapedat=DataFrame()
    clusmax=DataFrame()
            
    dv=divisors(Lx,Ly)
            
    clusmax=combine(groupby(res,:d_cutoff),:cluster=>maximum)
    
    rename!(clusmax,:cluster_maximum => :cm)

    for i in 1:(length(dv)-1)

        dat1=res1[res1.d_cutoff .==dv[i],:]
                
        wshape=append!(wshapedat,DataFrame(d_cutoff=dv[i],wshape=weibull_fit(dat1.cluster,dat1.freq)))

        fishdat=append!(fishdat,DataFrame(d_cutoff=dv[i],f=fish(dat1.cluster,dat1.freq)))

    end

    return innerjoin(fishdat,clusmax,wshapedat,on=:d_cutoff)

end
 


[ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for Cairo_jll [83423d85-b0ee-5818-9007-b63ccbeb887a]
│   exception = Required dependency Fontconfig_jll [a3f928ae-7b40-5064-980b-68af3947d34b] failed to load from a cache file.
└ @ Base loading.jl:1818
[ Info: Skipping precompilation since __precompile__(false). Importing Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80].
[ Info: Precompiling FFMPEG [c87230d0-a227-11e9-1b43-d7ebe4e7570a]
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for Cairo_jll [83423d85-b0ee-5818-9007-b63ccbeb887a]
│   exception = Required dependency Fontconfig_jll [a3f928ae-7b40-5064-980b-68af3947d34b] failed to load from a cache file.
└ @ Base loading.jl:1818
[ Info: Skipping precompilation since __precompile__(false). Importing FFMPEG [c87230d0-a227-11e9-1b43-d7ebe4e7570a].
[ Info: Precompiling FFMPEG_jll [b22a6f82-2f65-5046-a5b2-3

get_f_cm (generic function with 1 method)

In [ ]:
function weibull_fit(a,b)
        
        res=0.0
        
        try 
     
            res=shape(fit_mle(Weibull,wsample(a,b,100) .+ 1))
                
        catch err
            
            if isa(error, DomainError)
                
                res=0.0
                
            end
        end
        
        return res
        
end

function weibull2(a,b,binlength,m)
    
    res=0.0
            
        try 
        
            x=reduce(vcat,[fill(a[x1],b[x1]) for x1 in 1:length(a)])
            
            l = length(x)

            d,r=divrem(m,binlength)

            lns=fill(binlength,d)
            lns=vcat(lns,r)

            df=DataFrame(bins=reduce(vcat, [fill(i, v) for (i, v) in enumerate(lns)]),freq=0)

            df.vals = 1 .+ binlength .*(df.bins .- 1)

            [df.freq[x] += 1 for x in size(df)[1]]

            df=combine(groupby(df,:vals),:freq=> sum,renamecols=false)

            df=df[df.freq .>0,:]

            res=shape(fit_mle(Weibull,wsample(df.vals,df.freq,100)))
                
        catch err
            
            if isa(error, DomainError)
                
                res=0.0
                
            end
        end
        
        return res
end
        
function weibull3(a,b,binlength,m)
    
        
            x=reduce(vcat,[fill(a[i],b[i]) for i in 1:length(a)])
            
            l = length(x)

            d,r=divrem(m,binlength)

            lns=fill(binlength,d)
            lns=vcat(lns,r)

            df=DataFrame(bins=reduce(vcat, [fill(i, v) for (i, v) in enumerate(lns)]),freq=0)

            df.vals = 1 .+ binlength .*(df.bins .- 1)

            [df.freq[x] += 1 for x in size(df)[1]]

            df=combine(groupby(df,:vals),:freq=> sum,renamecols=false)

            df=df[df.freq .>0,:]

            res=shape(fit_mle(Weibull,wsample(df.vals,df.freq,100)))
        
            return res
end
        
function weibull_reg(a,b)
    
    b1= b ./ sum(b)
    b1= cumsum(b1)
    b2= log.(-log.(1 .- b1))
    a1=log.(a)

    df=DataFrame(X=a1,Y=b1)

    return coef(lm(@formula(Y~X),df))[2]
end

    
    
    

In [85]:
using EcologicalNetworksPlots

N = convert(BipartiteNetwork, web_of_life("M_PA_003"))

n = repeat(3:12, outer=20)
m = Array{Dict}(undef, length(n))

for i in eachindex(n)
  # Each run returns the network and its modules
  # We discard the network, and assign the modules to our object
  _, m[i] = n_random_modules(n[i])(N) |> x -> brim(x...)
end

Dict{String, Int64} with 39 entries:
  "Pseudomyrmex concolor"      => 1
  "Cecropia purpurascens"      => 2
  "Hirtella myrmecophila"      => 2
  "Allomerus octoarticulatus " => 2
  "Cordia nodosa"              => 1
  "Azteca isthmica"            => 2
  "Allomerus prancei"          => 3
  "Tococa bullifera"           => 3
  "Azteca schummani"           => 1
  "Pourouma heterophylla"      => 1
  "Crematogaster sp2 M_PA_001" => 3
  "Pseudomyrmex nigrescens"    => 1
  "Azteca sp1 M_PA_001"        => 1
  "Azteca sp4 M_PA_001"        => 3
  "Cecropia distachya"         => 2
  "Tachigali polyphylla"       => 1
  "Maieta guianensis"          => 3
  "Azteca sp5 M_PA_001"        => 1
  "Duroia saccifera"           => 1
  "Amaioua guianensis"         => 2
  "Cecropia concolor"          => 2
  "Solenops sp1 M_PA_001"      => 3
  "Azteca sp6 M_PA_001"        => 2
  "Azteca sp3 M_PA_001"        => 1
  "Pheidole minutula"          => 3
  ⋮                            => ⋮

In [46]:
Lx=32
Ly=32
abun=trunc(Int,Lx*Ly*0.1)

dat=DataFrame(x=sample(1:Ly,abun,replace=true),y=sample(1:Lx,abun,replace=true),pres=1)
    
    d=divisors(Lx,Ly)
    
    a=DataFrame(vec(collect(Iterators.product(1:Lx,1:Ly))))
    
    select!(a,"1"=>"x","2"=>"y")
    
    dat=outerjoin(dat,a, on= [:x=>:x,:y=>:y],order=:right)
    
    replace!(dat.pres,missing=>0)
    
    result=DataFrame()
    
    i=2
        
        cutx=ceil.(Int64,collect((1:Lx) ./ i))
        cuty=ceil.(Int64,collect((1:Ly) ./ i))
        
        dat1=deepcopy(dat)
        
        dat1[!,:x1] =cutx[dat1.x]
        dat1[!,:y1] = cuty[dat1.y]
        
        dat1=combine(groupby(dat1,[:x1,:y1]),:pres=>sum)
        
        mat1=unstack(dat1,:y1,:x1,:pres_sum)
        
        mat1=Matrix{Int64}(select(mat1,Not(:y1)))
        
        clust=zeros(Int64,size(mat1)[1],size(mat1)[2])
        
        largest=0
        
        lbl=collect(0:length(clust))
        
        rw=size(mat1)[1]
        cl=size(mat1)[2]
        
        for i2 in 1:cl 
            for i1 in 1:rw
                
                if mat1[i1,i2] > 0
                    
                    left= (i2-1)==0 ? 0 : clust[i1,i2-1]
                    above= (i1-1)==0 ? 0 : clust[i1-1,i2]
                    
                    if (left==0) && (above==0)
                        
                        largest+=1
                        clust[i1,i2]=largest
                        lbl[(rw*(i2-1))+i1]=largest
                        
                    else
                        
                        if (left!=0) && (above==0)
                            
                            clust[i1,i2]=clust[i1,i2-1]
                            lbl[(rw*(i2-1))+i1]=lbl[(rw*(i2-2))+i1]
                            
                        else
                            
                            if (left==0) && (above!=0)
                                
                                clust[i1,i2]=clust[i1-1,i2]
                                lbl[(rw*(i2-1))+i1]=lbl[(rw*(i2-1))+i1-1]
                                
                            else
                                
                                newlab=min(left,above)
                                lbl[findall(x-> x==left || x==above, lbl)] .=newlab
                                clust[i1,i2]=newlab
                                clust[findall(x-> x==max(left,above), clust)].=newlab
                            end
                        end
                    end
                end
            end
        end

        



In [79]:
using EcologicalNetworks

In [74]:
 centres=DataFrame()
        clusid=sort(unique(clust))
        for j in 2:(length(clusid))
            inds=findall(clust .==clusid[j])
            xc=mean([inds[x][1] for x in 1:length(inds)])
            yc=mean([inds[x][2] for x in 1:length(inds)])
            append!(centres,DataFrame(cluster=j,xc=xc,yc=yc))
        end

centres

Row,cluster,xc,yc
,Int64,Float64,Float64
1,2,4.54545,2.90909
2,3,11.0,1.0
3,4,13.7,2.2
4,5,10.0,3.5
5,6,6.0,5.0
6,7,14.0,5.0
7,8,5.0,6.0
8,9,12.5,6.0
9,10,16.0,6.0


In [76]:
pairwise(Euclidean(),[centres.xc centres.yc]')

32×32 Matrix{Float64}:
  0.0       6.73096   9.18197   5.48646  …  13.1023   13.539    14.5957
  6.73096   0.0       2.95466   2.69258     16.5529   15.2971   15.0
  9.18197   2.95466   0.0       3.92173     16.868    14.9308   14.0616
  5.48646   2.69258   3.92173   0.0         13.8654   12.659    12.5399
  2.54708   6.40312   8.19329   4.272       11.1803   11.1803   12.083
  9.68299   5.0       2.81603   4.272    …  14.8661   12.53     11.4018
  3.12415   7.81025   9.49368   5.59017     10.0499   10.4403   11.6619
  8.53396   5.22015   3.98497   3.53553     13.1244   10.9659   10.1119
 11.8642    7.07107   4.44185   6.5         15.6205   12.8062   11.1803
  5.91647  11.5569   13.1717    9.3039       8.00391   9.80115  11.877
 11.1808    7.82624   5.85491   6.36396  …  13.2004   10.3078    8.73212
  6.10785  10.0      11.0422    7.43303      7.07107   7.61577   9.21954
  7.44137   9.01388   9.26175   6.32456      7.63217   6.5       7.15891
  ⋮                                      ⋱ 

In [ ]:
pairwise(Euclidean(),)

In [ ]:
function perc_up2(dat::DataFrame,Lx::Int64,Ly::Int64)
    
    d=divisors(Lx,Ly)
    
    a=DataFrame(vec(collect(Iterators.product(1:Lx,1:Ly))))
    
    select!(a,"1"=>"x","2"=>"y")
    
    dat=outerjoin(dat,a, on= [:x=>:x,:y=>:y],order=:right)
    
    replace!(dat.pres,missing=>0)
    
    result=DataFrame()
    
    for i in d
        
        cutx=ceil.(Int64,collect((1:Lx) ./ i))
        cuty=ceil.(Int64,collect((1:Ly) ./ i))
        
        dat1=deepcopy(dat)
        
        dat1[!,:x1] =cutx[dat1.x]
        dat1[!,:y1] = cuty[dat1.y]
        
        dat1=combine(groupby(dat1,[:x1,:y1]),:pres=>sum)
        
        mat1=unstack(dat1,:y1,:x1,:pres_sum)
        
        mat1=Matrix{Int64}(select(mat1,Not(:y1)))
        
        clust=zeros(Int64,size(mat1)[1],size(mat1)[2])
        
        largest=0
        
        lbl=collect(0:length(clust))
        
        rw=size(mat1)[1]
        cl=size(mat1)[2]
        
        for i2 in 1:cl 
            for i1 in 1:rw
                
                if mat1[i1,i2] > 0
                    
                    left= (i2-1)==0 ? 0 : clust[i1,i2-1]
                    above= (i1-1)==0 ? 0 : clust[i1-1,i2]
                    
                    if (left==0) && (above==0)
                        
                        largest+=1
                        clust[i1,i2]=largest
                        lbl[(rw*(i2-1))+i1]=largest
                        
                    else
                        
                        if (left!=0) && (above==0)
                            
                            clust[i1,i2]=clust[i1,i2-1]
                            lbl[(rw*(i2-1))+i1]=lbl[(rw*(i2-2))+i1]
                            
                        else
                            
                            if (left==0) && (above!=0)
                                
                                clust[i1,i2]=clust[i1-1,i2]
                                lbl[(rw*(i2-1))+i1]=lbl[(rw*(i2-1))+i1-1]
                                
                            else
                                
                                newlab=min(left,above)
                                lbl[findall(x-> x==left || x==above, lbl)] .=newlab
                                clust[i1,i2]=newlab
                                clust[findall(x-> x==max(left,above), clust)].=newlab
                            end
                        end
                    end
                end
            end
        end
        
        #Get the pairwise smallest distances between clusters
        centres=DataFrame()
        clusid=sort(unique(clust))
        for j in 2:(length(clusid))
            inds=findall(clust .==clusid[j])
            xc=mean([inds[x][1] for x in 1:length(ids)])
            yc=mean([inds[x][2] for x in 1:length(ids)])
            append!(centres,DataFrame(cluster=j,xc=xc,yc=yc))
        end
            
            
        
        res=DataFrame(pres=reshape(mat1,length(mat1)),cluster=reshape(clust,length(clust)))
        res=combine(groupby(res,:cluster),:pres=>sum)
        res=res[res.cluster .!=0,:]

        freq=combine(groupby(res,:pres_sum),:pres_sum=>length)

        append!(result,DataFrame(d_cutoff=i,cluster=freq.pres_sum,freq=freq.pres_sum_length)) 
        
    end
    
    return result
end


In [ ]:
Lx=1000
Ly=500

#Get the f and cm metrics for the actual datasets

spdata=([ "alchco", "alsebl", "casear", "cecrin", "cordal", "crotbi", "jac1co",
        "luehse", "micoar", "paligu", "termam", "zantpr"],[271,9913,108,1381,223,634,309,
        199,652,1326,58,72],[2,4,2,1,7,3,22,9,52,6,84,1])

files=readdir("D://Project files//percolation//simdat//bci_spdat")

clusdat=DataFrame()

for i in 1:length(files)

    dat=CSV.read(string("D://Project files//percolation//simdat//bci_spdat//",files[i]), DataFrame)
    
    sp1=dat.sp[1]
    
    pop=spdata[2][findall(spdata[1].==dat.sp[1])]

    inds=reduce(vcat,[x .+ collect(0:((pop*8)[1])-1) for x in (1:(pop*40)[1]:nrow(dat))])

    #dat_real=dat[inds,:]
    
    dat=dat[Not(inds),:]
    
    #pars1=unique(dat_real[:,[:rep,:comp_dist,:disp,:k]])
    pars=unique(dat[:,[:rep,:comp_dist,:disp,:k]])
        
    for j in 1:size(pars)[1]
        
        rep1=pars.rep[j]
        comp_dist1=pars.comp_dist[j]
        disp1=pars.disp[j]
        k1=pars.k[j]

        dat1=dat[dat.rep .==rep1 .&& dat.comp_dist .==comp_dist1 .&& dat.disp .==disp1 .&& dat.k .==k1,[:x,:y,:pres]]
        
        res=perc_up(dat1,Lx,Ly)
            
        append!(clusdat,DataFrame(sp=sp1,rep=rep1,comp_dist=comp_dist1,disp=disp1,
                                    k=k1,d_cutoff=res.d_cutoff,cluster=res.cluster,
                                    freq=res.freq))
            
    end
end
    


In [ ]:
CSV.write("clusdat.csv",clusdat)

In [ ]:
pwd()

In [ ]:
res1=res[res.d_cutoff .==a,:]

m=sum(res1.cluster .* res1.freq)

bls=[2,5,10,15,20,50]

res2=zeros(Float64,6)

for i in 1:length(bls)
    
    res2[i]=weibull2(res1.cluster,res1.freq,i,m)
    
end




In [ ]:
shapedat=DataFrame()

a=unique(res.d_cutoff)

for j in a
    
    res1=res[res.d_cutoff .==a,:]

    x=reduce(vcat,[fill(res1.cluster[i],res1.freq[i]) for i in 1:size(res1)[1]])



In [ ]:

binlength=5

max1=271
l = length(x)

d,r=divrem(max1,binlength)

lns=fill(binlength,d)
lns=vcat(lns,r)

df=DataFrame(bins=reduce(vcat, [fill(i, v) for (i, v) in enumerate(lns)]),freq=0)

df.vals = 1 .+ binlength .*(df.bins .- 1)

[df.freq[i] += 1 for i in x]

df=combine(groupby(df,:vals),:freq=> sum,renamecols=false)

df=df[df.freq .>0,:]

shape(fit_mle(Weibull,wsample(df.vals,df.freq,100) .+ 1))

In [ ]:
res=perc_up(dat1,Lx,Ly)

a=DataFrame(vec(collect(Iterators.product(divisors(Lx,Ly),1:sum(dat1.pres)))))

select!(a,"1"=>"d_cutoff","2"=>"cluster")

res1=outerjoin(res,a, on= [:d_cutoff=>:d_cutoff,:cluster=>:cluster],order=:right)

replace!(res1.freq,missing=>0)
    
res1.freq =identity.(res1.freq)

fishdat=DataFrame()
wshapedat=DataFrame()
clusmax=DataFrame()
            
dv=divisors(Lx,Ly)
            
clusmax=combine(groupby(res,:d_cutoff),:cluster=>maximum)
    
rename!(clusmax,:cluster_maximum => :cm)


In [ ]:
i=67

dat1=res1[res1.d_cutoff .==dv[i],:]
                



In [ ]:
weibull_fit(dat1.cluster,dat1.freq)


In [ ]:
dat1

In [ ]:

dat.freq = dat.freq ./ sum(dat.freq)

dat.freq = log.(.- log.(1 .- dat.freq))

dat.cluster=log.(dat.cluster)

ols = lm(@formula(freq ~ cluster), dat)

coef(ols)

In [ ]:
#Get the inflection points

    f_dash=segmented_f(met_res,divisors(Lx,Ly))[1]
    f_dash_var=segmented_f(met_res,divisors(Lx,Ly))[2]

    cm_dash=segmented_cm(met_res,divisors(Lx,Ly))[1]
    cm_dash_var=segmented_cm(met_res,divisors(Lx,Ly))[2] 
    
    return f_dash,f_dash_var,cm_dash,cm_dash_var

In [ ]:
Lx=1000
Ly=500

#Get the f and cm metrics for the actual datasets

spdata=([ "alchco", "alsebl", "casear", "cecrin", "cordal", "crotbi", "jac1co",
        "luehse", "micoar", "paligu", "termam", "zantpr"],[271,9913,108,1381,223,634,309,
        199,652,1326,58,72],[2,4,2,1,7,3,22,9,52,6,84,1])

files=readdir("D://Project files//percolation//simdat//bci_spdat")

    
    dat=CSV.read(string("D://Project files//percolation//simdat//bci_spdat//",files[1]), DataFrame)
    
    sp1=dat.sp[1]
    
    pop=spdata[2][findall(spdata[1].==dat.sp[1])]

    inds=reduce(vcat,[x .+ collect(0:((pop*8)[1])-1) for x in (1:(pop*40)[1]:nrow(dat))])

    #dat_real=dat[inds,:]
    
    dat=dat[Not(inds),:]
    
    #pars1=unique(dat_real[:,[:rep,:comp_dist,:disp,:k]])
    pars=unique(dat[:,[:rep,:comp_dist,:disp,:k]])
    
    resdat=DataFrame()

#for i in 1:size(pars)[1]
    
    i=2
        
        rep1=pars.rep[i]
        comp_dist1=pars.comp_dist[i]
        disp1=pars.disp[i]
        k1=pars.k[i]

        dat1=dat[dat.rep .==rep1 .&& dat.comp_dist .==comp_dist1 .&& dat.disp .==disp1 .&& dat.k .==k1,[:x,:y,:pres]]
        
        res=perc_up(dat1,Lx,Ly)
        
        #append!(resdat,DataFrame(rep=rep1,comp_dist=comp_dist1,disp=disp1,k=k1,d_cutoff=res.d_cutoff,f=res.f,cm=res.cm,wshape=res.wshape))
#end

In [ ]:
#res=perc_up(dat1,Lx,Ly)

dat2=res[res.d_cutoff .==1,:]

plot(dat2.cluster,dat2.freq)

In [ ]:
dat3

In [ ]:
dat3=resdat[1:67,:]

plot(dat3.d_cutoff,dat3.wshape)

In [ ]:
    a=DataFrame(vec(collect(Iterators.product(divisors(Lx,Ly),1:sum(dat1.pres)))))

    select!(a,"1"=>"d_cutoff","2"=>"cluster")

    res=outerjoin(res,a, on= [:d_cutoff=>:d_cutoff,:cluster=>:cluster],order=:right)

    replace!(res.freq,missing=>0)
    
    res.freq =identity.(res.freq)

    dv=divisors(Lx,Ly)
    
    i=2
    
    dat=res[res.d_cutoff .==dv[i],:]
    
    weibull_fit(dat.cluster,dat.freq)

In [ ]:
i=2
    
    dat=res[res.d_cutoff .==dv[i],:]
    
    weibull_fit(dat.cluster,dat.freq)

In [ ]:
shape(fit_mle(Weibull,wsample(dat.cluster,dat.freq,200)))

In [ ]:
wsample(dat.cluster,)

In [ ]:
#a=DataFrame(vec(collect(Iterators.product(divisors(Lx,Ly),1:sum(dat1.pres)))))
#select!(a,"1"=>"d_cutoff","2"=>"cluster")

#res=outerjoin(res,a, on= [:d_cutoff=>:d_cutoff,:cluster=>:cluster],order=:right)

#res.freq=replace(res.freq,missing=>0)
res1=res[res.d_cutoff .==1,:]

samp=vcat(fill.(res1.cluster, res1.freq)...)

shapedat=zeros(Float64,100)

for j in 1:100
    shapedat[j]=shape(fit_mle(Weibull, samp, alpha0=1, maxiter= 1000, tol= 1e-16))
    

for i in unique(res.d_cutoff)
    
    res1=res[res.d_cutoff .==i,:]
        
end


In [ ]:
dv=divisors(Lx,Ly)
fishdat=DataFrame()
clusmax=DataFrame()

for i in dv

dat1=res[res.d_cutoff .==i,:]

fishdat=append!(fishdat,DataFrame(d_cutoff=i,f=fish(dat1.cluster,dat1.freq)))

dat1=dat1[dat1.freq.>0,:]

clusmax=append!(clusmax,DataFrame(d_cutoff=i,cm=maximum(dat1.cluster)))
    
end

In [ ]:
met_res=innerjoin(fishdat,clusmax,on=:d_cutoff)

In [ ]:
#Analyse bci sp files
using CSV, DataFrames, Random, LinearAlgebra, Distributions, Distances, CategoricalArrays

Lx=1000
Ly=500

#Get the f and cm metrics for the actual datasets

spdata=([ "alchco", "alsebl", "casear", "cecrin", "cordal", "crotbi", "jac1co",
        "luehse", "micoar", "paligu", "termam", "zantpr"],[271,9913,108,1381,223,634,309,
        199,652,1326,58,72],[2,4,2,1,7,3,22,9,52,6,84,1])

files=readdir("D://Project files//percolation//simdat//bci_spdat")

result=DataFrame()

for i in 1:length(files)
    
    dat=CSV.read(string("D://Project files//percolation//simdat//bci_spdat//",files[i]), DataFrame)
    
    sp1=dat.sp[1]
    
    pop=spdata[2][findall(spdata[1].==dat.sp[i])]

    inds=reduce(vcat,[x .+ collect(0:((pop*8)[1])-1) for x in (1:(pop*40)[1]:nrow(dat))])

    #dat_real=dat[inds,:]
    
    dat=dat[Not(inds),:]
    
    #pars1=unique(dat_real[:,[:rep,:comp_dist,:disp,:k]])
    pars=unique(dat[:,[:rep,:comp_dist,:disp,:k]])

    for i in 1:nrow(pars)

        rep1=pars.rep[i]
        comp_dist1=pars.comp_dist[i]
        disp1=pars.disp[i]
        k1=pars.k[i]

        dat1=dat[dat.rep .==rep1 .&& dat.comp_dist .==comp_dist1 .&& dat.disp .==disp1 .&& dat.k .==k1,[:x,:y,:pres]]

        res=get_f_cm(dat1,Lx,Ly)

        append!(result,DataFrame(sp=sp1,rep=rep1,comp_dist=comp_dist1,disp=disp1,k=k1,f=res[1],f_dev=res[2],cm=res[3],cm_dev=res[4]))
        
    end
end



In [ ]:
sps=unique(result.sp)

obs=DataFrame()

bcidat=CSV.read("C://Users//mihir//projects//percolation//rawdat//bci_select.csv",DataFrame)
bcidat=bcidat[:,[:sp,:x,:y,:pres]]

for i in sps
    
    dat=bcidat[bcidat.sp .==i,[:x,:y,:pres]]
    
    dat.x=ceil.(Int64,dat.x)
    dat.y=ceil.(Int64,dat.y)
    
    res=get_f_cm(dat,Lx,Ly)
    
    append!(obs,DataFrame(sp=i,f_obs=res[1],f_var_obs=res[2],cm_obs=res[3],cm_var_obs=res[4]))
end
    

result=innerjoin(result,obs,on=:sp)

CSV.write("C://Users//mihir//projects//percolation//results//bci_met_res.csv",result)  

In [ ]:
#Misc functions

function add_breakpoint(data, bp)
	data[!, "since_bp"] = max.(0, data[!, "d_cutoff"] .- bp);
end

function fit_piecewise_f(data, minbp, maxbp)
  min_deviance = Inf
  best_model = nothing
  best_bp = 0
  current_model = nothing
  
  for bp in minbp:maxbp
    add_breakpoint(data, bp)
    current_model = lm(@formula(f ~ d_cutoff + since_bp), data)
    if deviance(current_model) < min_deviance
      min_deviance = deviance(current_model)
      best_model = current_model
      best_bp = bp
    end
  end
  
  return best_bp,min_deviance
end

function segmented_f(dat::DataFrame,d::Vector{Int64})
    
    min_dev = Inf
    best_bp = 0
    current1=current2=nothing
    
    for bp in 3:(length(d)-2)
        
        current1=lm(@formula(f~d_cutoff),dat[1:bp,:])
        current2=lm(@formula(f~d_cutoff),dat[(bp+1):length(d),:])
        
        if (deviance(current1)+deviance(current2)) < min_dev
            
            min_dev= deviance(current1)+deviance(current2)
            best_bp=bp
        end
    end
    
    return dat.d_cutoff[best_bp],min_dev
    
end

function segmented_cm(dat::DataFrame,d::Vector{Int64})
    
    min_dev = Inf
    best_bp = 0
    current1=current2=nothing
    
    for bp in 3:(length(d)-2)
        
        current1=lm(@formula(cm~d_cutoff),dat[1:bp,:])
        current2=lm(@formula(cm~d_cutoff),dat[(bp+1):length(d),:])
        
        if (deviance(current1)+deviance(current2)) < min_dev
            
            min_dev= deviance(current1)+deviance(current2)
            best_bp=bp
        end
    end
    
    return dat.d_cutoff[best_bp],min_dev
    
end
    
    
                
    

In [ ]:
#Validation code
#Goal:To demonstrate that the f1 and cm1 metrices robustly represent the signatures of different dispersion mechanisms
#To do: create an example (seed =1)




In [ ]:
using CSV, DataFrames, Random, LinearAlgebra, Distributions, Distances, CategoricalArrays

#Declare the functions

#Kill #ndeath individuals with the prob equal to habitat suitability
function kill!(A::Matrix{Int8}, B::Matrix{Float64}, ndeath::Int64)

    samp=findall(A.==1)
    A[wsample(samp,B[samp],ndeath,replace=false)].=0
    return nothing
end

#Calculate recruitment probability as a function of dispersal kernel and neighborhood crowding effect
function recprob(a::CartesianIndex{2},A::Matrix{Int8},disps::Int64,d::Int64,
        b::Vector{Float64})

    xs=((a[1]-disps):(a[1]+disps))
    xs=xs[0 .<xs .<=Ly]

    ys=(a[2]-disps):(a[2]+disps)
    ys=ys[0 .<ys .<=Lx]

    prob=0

    if sum(A[xs,ys])>0

        parents=findall(A[xs,ys].==1)

        for i in 1:length(parents)

            prob+=pdf(Normal(0,disps/2),euclidean([a[1],a[2]],[xs[parents[i][1]],ys[parents[i][2]]]))
        end

        xc=((a[1]-d):(a[1]+d))
        xc=xc[0 .<xc .<=Ly]

        yc=(a[2]-d):(a[2]+d)
        yc=yc[0 .<yc .<=Lx]

        prob*=b[sum(A[xc,yc])+1]
    end

    return prob

end


function spawns!(A::Matrix{Int8},disps::Int64,comp_dist::Int64,comp_list::Vector{Float64},nbirth::Int64)

    samp2=shuffle(findall(A.==0))

    probs=map(x->recprob(x,A,disps,comp_dist,comp_list),samp2)

    A[wsample(samp2,probs,nbirth,replace=false)].=1

    return nothing

end


function sampsim!(A::Matrix{Int8},B::Matrix{Float64},disps::Int64,comp_dist::Int64, comp_list::Vector{Float64},nbirth::Int64,ndeath::Int64)

    for i in 1:100

        kill!(A,B,ndeath)
        spawns!(A,disps,comp_dist,comp_list,nbirth)

    end

    return nothing
end


In [ ]:
using CSV, DataFrames, Random, LinearAlgebra, Distributions, Distances, CategoricalArrays

#Declare the functions

#Kill #ndeath individuals with the prob equal to habitat suitability
function kill!(A::Matrix{Int8}, B::Matrix{Float64}, ndeath::Int64)

    samp=findall(A.==1)
    A[wsample(samp,B[samp],ndeath,replace=false)].=0
    return nothing

end

#Calculate recruitment probability as a function of dispersal kernel and neighborhood crowding effect
function recprob(a::CartesianIndex{2},A::Matrix{Int8},disps::Int64,d::Int64,
        b::Vector{Float64})

    xs=((a[1]-disps):(a[1]+disps))
    xs=xs[0 .<xs .<=Ly]

    ys=(a[2]-disps):(a[2]+disps)
    ys=ys[0 .<ys .<=Lx]

    prob=0

    if sum(A[xs,ys])>0

        parents=findall(A[xs,ys].==1)

        for i in 1:length(parents)

            prob+=pdf(Normal(0,disps/2),euclidean([a[1],a[2]],[xs[parents[i][1]],ys[parents[i][2]]]))
        end

        xc=((a[1]-d):(a[1]+d))
        xc=xc[0 .<xc .<=Ly]

        yc=(a[2]-d):(a[2]+d)
        yc=yc[0 .<yc .<=Lx]

        prob*=b[sum(A[xc,yc])+1]
    end

    return prob

end


function spawns!(A::Matrix{Int8},disps::Int64,comp_dist::Int64,comp_list::Vector{Float64},nbirth::Int64)

    samp2=shuffle(findall(A.==0))

    probs=map(x->recprob(x,A,disps,comp_dist,comp_list),samp2)

    A[wsample(samp2,probs,nbirth,replace=false)].=1

    return nothing

end


function sampsim!(A::Matrix{Int8},B::Matrix{Float64},disps::Int64,comp_dist::Int64, comp_list::Vector{Float64},nbirth::Int64,ndeath::Int64)

    for i in 1:80

        kill!(A,B,ndeath)
        spawns!(A,disps,comp_dist,comp_list,nbirth)

    end

    return nothing
end

#Set up parameters

num= Base.parse(Int, ENV["SLURM_ARRAY_TASK_ID"])

Lx=1000
Ly=500

spdata=([ "alchco", "alsebl", "casear", "cecrin", "cordal", "crotbi", "jac1co",
        "luehse", "micoar", "paligu", "termam", "zantpr"],[271,9913,108,1381,223,634,309,
        199,652,1326,58,72],[2,4,2,1,7,3,22,9,52,6,84,1])

#Load the demography data
demo_data=CSV.read(string("/gpfs/home/mumarani/perc.demodata.bci.csv"),DataFrame)

comp_dists=[15,30]
ks=[-20,-1,1,20]

disps=[spdata[3][num],2,5,10,20]

pars=collect(Iterators.product(comp_dists,ks,disps))

abun=spdata[2][num]

demodat=demo_data[demo_data.sp .== spdata[1][num],:]

nbirth=ndeath=abs(ceil(Int64,rand(Normal(demodat[1,3],demodat[1,4]))))


result=DataFrame()

 for i1 in 1:5

        landscape=CSV.read(string("/gpfs/home/mumarani/landscapes2/",readdir("/gpfs/home/mumarani/landscapes2")[i1]), DataFrame)
        repl=landscape.rep[1]
        rangepar=landscape.rangepars[1]
        landscape[:,:pres]=shuffle([ones(Int8,abun);zeros(Int8,(nrow(landscape)-abun))])

        initial=filter(:pres => x-> x .== 1,landscape)
        initial=initial[:,[:rep,:rangepars,:x,:y,:pres]]

        init=unstack(landscape[:,["y","x","pres"]],:y,:x,:pres)
        init=Matrix{Int8}(init[:,Not("y")])

        habmat=unstack(landscape[:,["y","x","soiltype"]],:y,:x,:soiltype)
        habmat=Matrix{Float64}(habmat[:,Not("y")])

        popmat=deepcopy(init)

        for i2 in 1:length(pars)

            comp_dist=pars[i2][1]
            k=pars[i2][2]
            disp=pars[i2][3]

            maxnb=((2*comp_dist+1)^2)-1
            comp_list=map(y-> 1/(1+exp(k*(-y))),1:maxnb)
            comp_list=(comp_list .- minimum(comp_list) .+ 0.1)/(maximum(comp_list)-minimum(comp_list) .+ 0.1)

            sampsim!(popmat,habmat,disp,comp_dist,comp_list,nbirth,ndeath)

            popmat1=DataFrame(popmat,:auto)
            rename!(x-> strip(string(x), ['x']),popmat1)
            popmat1[:,:y].=1:Ly
            popmat1=stack(popmat1,1:Lx)
            rename!(popmat1,:variable=> :x, :value => :pres)
            popmat1[!,:x] = parse.(Int64,popmat1[!,:x])
            filter!(:pres => x-> x .== 1,popmat1)
            popmat1[:,:rep].=repl
            popmat1[:,:rangepar].=rangepar
            popmat1[:,:disp].=disp
            popmat1[:,:k].=k
            popmat1[:,:sp].=spdata[1][num]
            append!(result,popmat1)
        end
    end
    CSV.write(string(spdata[1][num],".csv"),result)
    
    


In [ ]:
using CSV, DataFrames, Random, LinearAlgebra, Distributions, Distances

#Declare the functions

#Kill #ndeath individuals with the prob equal to habitat suitability
function kill!(A::Matrix{Int8}, B::Matrix{Float64}, ndeath::Int64)

    samp=findall(A.==1)
    A[wsample(samp,B[samp],ndeath,replace=false)].=0
    return nothing

end

#Calculate recruitment probability as a function of dispersal kernel and neighborhood crowding effect
function recprob(a::CartesianIndex{2},A::Matrix{Int8},disps::Int64,d::Int64,
        b::Vector{Float64})

    xs=((a[1]-disps):(a[1]+disps))
    xs=xs[0 .<xs .<=Ly]

    ys=(a[2]-disps):(a[2]+disps)
    ys=ys[0 .<ys .<=Lx]

    prob=0

    if sum(A[xs,ys])>0

        parents=findall(A[xs,ys].==1)

        for i in 1:length(parents)

            prob+=pdf(Normal(0,disps/2),euclidean([a[1],a[2]],[xs[parents[i][1]],ys[parents[i][2]]]))
        end

        xc=((a[1]-d):(a[1]+d))
        xc=xc[0 .<xc .<=Ly]

        yc=(a[2]-d):(a[2]+d)
        yc=yc[0 .<yc .<=Lx]

        prob*=b[sum(A[xc,yc])+1]
    end

    return prob

end


function spawns!(A::Matrix{Int8},disps::Int64,comp_dist::Int64, comp_list::Vector{Float64},nbirth::Int64)

    samp2=shuffle(findall(A.==0))

    probs=map(x->recprob(x,A,disps,comp_dist,comp_list),samp2)

    A[wsample(samp2,probs,nbirth,replace=false)].=1

    return nothing

end


function sampsim!(A::Matrix{Int8},B::Matrix{Float64},disps::Int64,comp_dist::Int64, comp_list::Vector{Float64},nbirth::Int64,ndeath::Int64)

    for i in 1:80

        kill!(A,B,ndeath)
        spawns!(A,disps,comp_dist,comp_list,nbirth)

    end

    return nothing
end

In [ ]:
Lx=1000
Ly=500

spdata=([ "alchco", "alsebl", "casear", "cecrin", "cordal", "crotbi", "jac1co",
        "luehse", "micoar", "paligu", "termam", "zantpr"],[271,9913,108,1381,223,634,309,
        199,652,1326,58,72],[2,4,2,1,7,3,22,9,52,6,84,1])

demo_data=CSV.read("C:/Users/Mihir/Documents/perc.demodata.bci.csv",DataFrame)

comp_dists=[15,30]
ks=[-20,-1,1,20]

disps=[spdata[3][1],2,5,10,20]

pars=collect(Iterators.product(comp_dists,ks,disps))

abun=spdata[2][1]

demodat=demo_data[demo_data.sp .== spdata[1][1],:]

nbirth=ndeath=abs(ceil(Int64,rand(Normal(demodat[1,3],demodat[1,4]))))

landscape=CSV.read("C:/Users/Mihir/Documents/landscapes/landscape_201.csv", DataFrame)
        repl=landscape.rep[1]
        rangepar=landscape.rangepars[1]
        landscape[:,:pres]=shuffle([ones(Int8,abun);zeros(Int8,(nrow(landscape)-abun))])

        initial=filter(:pres => x-> x .== 1,landscape)
        initial=initial[:,[:rep,:rangepars,:x,:y,:pres]]

        init=unstack(landscape[:,["y","x","pres"]],:y,:x,:pres)
        init=Matrix{Int8}(init[:,Not("y")])

        habmat=unstack(landscape[:,["y","x","soiltype"]],:y,:x,:soiltype)
        habmat=Matrix{Float64}(habmat[:,Not("y")])

        popmat=deepcopy(init)

In [ ]:
i2=1 

comp_dist=pars[i2][1]
            k=pars[i2][2]
            disp=pars[i2][3]

            maxnb=((2*comp_dist+1)^2)-1
            comp_list=map(y-> 1/(1+exp(k*(-y))),1:maxnb)
            comp_list=(comp_list .- minimum(comp_list))/(maximum(comp_list)-minimum(comp_list))

In [ ]:
disp

In [ ]:
#Validation code
#Simulate the population dynamics for large number of replicates (for each param combo),
#calculate f1 and cm1 metrics

In [ ]:
using CSV, DataFrames, Random, LinearAlgebra, Distributions, Distances

#Declare the functions
 
#Kill #ndeath individuals with the prob equal to habitat suitability 
function kill!(A::Matrix{Int8}, B::Matrix{Float64}, ndeath::Int64)
    
    samp=findall(A.==1)
    A[wsample(samp,B[samp],ndeath,replace=false)].=0
    return nothing

end

#Calculate recruitment probability as a function of dispersal kernel and neighborhood crowding effect 
function recprob(a::CartesianIndex{2},A::Matrix{Int8},disps::Int64,d::Int64,method::Int64,
        b::Tuple{Vector{Float64}, Vector{Float64}})
    
    xs=((a[1]-disps):(a[1]+disps))
    xs=xs[0 .<xs .<=Ly]

    ys=(a[2]-disps):(a[2]+disps)
    ys=ys[0 .<ys .<=Lx]
    
    prob=0
   
    if sum(A[xs,ys])>0 
        
        parents=findall(A[xs,ys].==1)

        for i in 1:length(parents)
            
            prob+=pdf(Normal(0,disps/2),euclidean([a[1],a[2]],[xs[parents[i][1]],ys[parents[i][2]]]))
        end

        xc=((a[1]-d):(a[1]+d))
        xc=xc[0 .<xc .<=Ly]

        yc=(a[2]-d):(a[2]+d)
        yc=yc[0 .<yc .<=Lx]
        
        prob*=b[method][sum(A[xc,yc])+1]
    end
    
    return prob
    
end


function spawns!(A::Matrix{Int8},disps::Int64,comp_dist::Int64,method::Int64,
        comp_list::Tuple{Vector{Float64}, Vector{Float64}},nbirth::Int64)  
    
    samp2=shuffle(findall(A.==0))
    
    probs=map(x->recprob(x,A,disps,comp_dist,method,comp_list),samp2)
    
    A[wsample(samp2,probs,nbirth,replace=false)].=1
    
    return nothing
    
end
    

function sampsim!(A::Matrix{Int8},B::Matrix{Float64},disps::Int64,comp_dist::Int64,method::Int64,
        comp_list::Tuple{Vector{Float64}, Vector{Float64}},nbirth::Int64,ndeath::Int64) 
    
    for i in 1:100
        
        kill!(A,B,ndeath)
        spawns!(A,disps,comp_dist,method,comp_list,nbirth)
        
    end
    
    return nothing
end

In [ ]:
Lx=1000 
Ly=500


spdata=([ "alchco", "alsebl", "casear", "cecrin", "cordal", "crotbi", "jac1co", 
        "luehse", "micoar", "paligu", "termam", "zantpr"],[271,9913,108,1381,223,634,309,
        199,652,1326,58,72],[2,4,2,1,7,3,22,9,52,6,84,1])

#Load the demography data
demo_data=CSV.read(string("C:/Users/mihir/Documents/perc.demodata.bci.csv"),DataFrame)

inds=[1,2,3,4,5,101,102,103,104,105,201,202,203,204,205]

comp_dists=[15,30]
methodrefs=DataFrame(ind=[1, 2], methods=["positive","negative"])
pars=collect(Iterators.product([15,30],[1,2]))

cd("C:\\Users\\mihir\\Documents\\landscapes")

bigres=DataFrame()

for i in 9:length(spdata[1])
    
    disp=spdata[3][i]
    
    abun=spdata[2][i]
    
    demodat=demo_data[demo_data.sp .== spdata[1][i],:]
    
    nbirth=ndeath=abs(ceil(Int64,rand(Normal(demodat[1,3],demodat[1,4]))))
    
    result=DataFrame()

    for i1 in inds

        landscape=CSV.read(string("landscape_",i1,".csv"), DataFrame)
        repl=landscape.rep[1] 
        rangepar=landscape.rangepars[1]    
        landscape[:,:pres]=shuffle([ones(Int8,abun);zeros(Int8,(nrow(landscape)-abun))])

        initial=filter(:pres => x-> x .== 1,landscape)
        initial=initial[:,[:rep,:rangepars,:x,:y,:pres]]

        init=unstack(landscape[:,["y","x","pres"]],:y,:x,:pres)
        init=Matrix{Int8}(init[:,Not("y")])

        habmat=unstack(landscape[:,["y","x","soiltype"]],:y,:x,:soiltype)
        habmat=Matrix{Float64}(habmat[:,Not("y")])

        popmat=deepcopy(init)
        
        for i2 in 1:length(pars)
        
            comp_dist=pars[i2][2]
            method=pars[i2][2]

            maxnb=((2*comp_dist+1)^2)-1
            neigh=log.(maxnb,1:maxnb)
            comp_list=([0.1; 0.1 .+ neigh],[1.0;1.0 .- neigh])

            sampsim!(popmat,habmat,disp,comp_dist,method,comp_list,nbirth,ndeath)

            popmat1=DataFrame(popmat,:auto)
            rename!(x-> strip(string(x), ['x']),popmat1)
            popmat1[:,:y].=1:Ly
            popmat1=stack(popmat1,1:Lx)
            rename!(popmat1,:variable=> :x, :value => :pres)
            popmat1[!,:x] = parse.(Int64,popmat1[!,:x])
            filter!(:pres => x-> x .== 1,popmat1)
            popmat1[:,:rep].=repl 
            popmat1[:,:rangepar].=rangepar
            popmat1[:,:disp].=disp
            popmat1[:,:method].=method
            popmat1[:,:sp].=spdata[1][i]
            append!(result,popmat1)
            append!(bigres,popmat1)
        end
    end
    CSV.write(string(spdata[1][i],".csv"),result)
end
CSV.write(string("bci_perc_null",".csv"),result)
        

        

In [ ]:
?CSV.write

In [ ]:
demo_data=CSV.read(string("C:/Users/mihir/Documents/perc.demodata.bci.csv"),DataFrame)
demo_data=demo_data[Not(3),:]

In [ ]:
i1=1
i2=1
abun=1000


landscape=CSV.read(string("landscape_",i1,".csv"), DataFrame)
        landscape[:,:pres]=shuffle([ones(Int8,abun);zeros(Int8,(nrow(landscape)-abun))])

        initial=filter(:pres => x-> x .== 1,landscape)
        initial=initial[:,[:rep,:rangepars,:x,:y,:pres]]

        init=unstack(landscape[:,["y","x","pres"]],:y,:x,:pres)
        init=Matrix{Int8}(init[:,Not("y")])

        habmat=unstack(landscape[:,["y","x","soiltype"]],:y,:x,:soiltype)
        habmat=Matrix{Float64}(habmat[:,Not("y")])
            
        comp_dist=pars[i2][2]
        method=pars[i2][2]

        maxnb=((2*comp_dist+1)^2)-1
        neigh=log.(maxnb,1:maxnb)
        comp_list=([0.1; 0.1 .+ neigh],[1.0;1.0 .- neigh])
  

        popmat=deepcopy(init)

In [ ]:
using CSV, DataFrames, Random, LinearAlgebra, Distributions, Distances

#Declare the functions
 
#Kill #ndeath individuals with the prob equal to habitat suitability 
function kill!(A::Matrix{Int8}, B::Matrix{Float64}, ndeath::Int64)
    
    samp=findall(A.==1)
    A[wsample(samp,B[samp],ndeath,replace=false)].=0
    return nothing

end

#Calculate recruitment probability as a function of dispersal kernel and neighborhood crowding effect 
function recprob(a::CartesianIndex{2},A::Matrix{Int8},disps::Int64,d::Int64,method::Int64,
        b::Tuple{Vector{Int64}, Vector{Float64}, Vector{Float64}})
    
    xs=((a[1]-disps):(a[1]+disps))
    xs=xs[0 .<xs .<=Ly]

    ys=(a[2]-disps):(a[2]+disps)
    ys=ys[0 .<ys .<=Lx]
    
    prob=0
   
    if sum(A[xs,ys])>0 
        
        parents=findall(popmat[xs,ys].==1)

        for i in 1:length(parents)
            
            prob+=pdf(Normal(0,disps/2),euclidean([a[1],a[2]],[xs[parents[i][1]],ys[parents[i][2]]]))
        end

        xc=((a[1]-d):(a[1]+d))
        xc=xc[0 .<xc .<=Ly]

        yc=(a[2]-d):(a[2]+d)
        yc=yc[0 .<yc .<=Lx]
        
        prob*=comp_list[method][sum(A[xc,yc])+1]
    end
    
    return prob
    
end


function spawns!(A::Matrix{Int8},disps::Int64,comp_dist::Int64,method::Int64,
        comp_list::Tuple{Vector{Int64}, Vector{Float64}, Vector{Float64}},nbirth::Int64)  
    
    samp2=shuffle(findall(A.==0))
    
    probs=map(x->recprob(x,A,disps,comp_dist,method,comp_list),samp2)
    
    A[wsample(samp2,probs,nbirth,replace=false)].=1
    
    return nothing
    
end
    

function sampsim!(A::Matrix{Int8},B::Matrix{Float64},disps::Int64,comp_dist::Int64,method::Int64,
        comp_list::Tuple{Vector{Int64}, Vector{Float64}, Vector{Float64}},nbirth::Int64,ndeath::Int64) 
    
    for i in 1:100
        
        kill!(A,B,ndeath)
        spawns!(A,disps,comp_dist,method,comp_list,nbirth)
        
    end
    
    return nothing
end


In [ ]:
using  CSV, DataFrames, Random, LinearAlgebra, Distributions

d_cutoffs=[[1,2];collect(5:5:100)]
Lx=1000
Ly=500

files=readdir("C:\\Users\\mihir\\Documents\\landscapes")
finals=files[occursin.("final",files)]
initials=files[occursin.("ini tial",files)]
landscapes=files[occursin.("landscape",files)]

results=DataFrame()

for i in 1:length(finals)
    
    data=CSV.read(string("C:/Users/mihir/Documents/landscapes/",finals[i]),DataFrame)
    
    param=unique(data[:,[:rep,:rangepar,:disp,:method]])
    
    for j in 1:nrow(param)
        
        dat= data[(data.rep .==param[j,1]) .& (data.rangepar .==param[j,2]) .& 
            (data.disp .==param[j,3]) .& (data.method .==param[j,4]),:]
        
        for k in d_cutoffs
            
            dat2=dat
            
            dat2[:,:x]=map(x -> findlast(x .>= collect(1:k:Lx)), dat2[:,:x])
            
            dat2[:,:y]=map(x -> findlast(x .>= collect(1:k:Ly)), dat2[:,:y])
            
            dat1=zeros(Int8,ceil(Int,Ly/k),ceil(Int,Lx/k))
            
            for k1 in 1:nrow(dat2)
                
                dat1[dat2[k1,:y],dat2[k1,:x]]=1
            end
            
            clust=zeros(Int,size(dat1))
            
            largest=0
            
            lbl=collect(0:length(clust))
            
            for i1 in 1:size(dat1)[1], i2 in 1:size(dat1)[2]
                
                if(dat1[i1,i2]>0)
                    
                    left=i2>1 ? clust[i1,i2-1] : 0
                    above=i1>1 ? clust[i1-1,i2] : 0
                    
                    if(left==0 && above==0)
                        
                        largest+=1
                        clust[i1,i2]=largest
                        lbl[(10*(i2-1))+i1]=largest
                    
                    elseif(left!=0 && above==0)
                        
                        clust[i1,i2]=clust[i1,i2-1]
                        lbl[(10*(i2-1))+i1]=lbl[(10*(i2-2))+i1]
                    
                    elseif(left==0 && above!=0)
                        
                        clust[i1,i2]=clust[i1-1,i2]
                        lbl[(10*(i2-1))+i1]=lbl[(10*(i2-1))+i1-1]
                    
                    else
                        
                        newlab=min(left,above)
                        clust[i1,i2]=newlab
                        clust[findall(clust.==max(left,above))].=newlab
                        lbl[findall(lbl.==max(left,above))].=newlab
                    end
                end
            end
            
            clus=unique(clust)
            clus=clus[clus.>0]
            cnts=[count(i->i.==x, clust) for x=clus]
            c_size=unique(cnts)
            cnts=[count(i->i.==x,cnts) for x=c_size]
            ln=length(c_size)
            
            res=DataFrame(rep=repeat([param[j,1]],ln),rangepar=repeat([param[j,2]],ln),
                disp=repeat([param[j,3]],ln),method=repeat([param[j,4]],ln),
                d_cutoff=repeat([k],ln),
                cluster=c_size,counts=cnts)
            
            append!(results,res)
        end
    end
end
            
            
            

       
            

In [ ]:
using  CSV, DataFrames, Random, LinearAlgebra, Distributions, Plots

Lx=1000 
Ly=500
abun=1000
timepoints=10000
nbirth=ndeath=convert(Int64,floor(abun*0.1))

disp=5
methodrefs=DataFrame(ind=[1, 2, 3], methods=["neutral","positive","negative"])
pars=collect(Iterators.product(disp,methodrefs[:,:ind]))

cd("C:\\Users\\mihir\\Documents\\landscapes")

landscape=CSV.read(string("landscape_",1,".csv"), DataFrame)

rangepar=landscape.rangepars[1]
      
landscape[:,:pres]=shuffle([ones(Int8,abun);zeros(Int8,(nrow(landscape)-abun))])

initial=filter(:pres => x-> x .== 1,landscape)

scatter(initial.x,initial.y)

popmat=unstack(landscape[:,["y","x","pres"]],:y,:x,:pres)
popmat=Matrix(popmat[:,Not("y")])

habmat=unstack(landscape[:,["y","x","soiltype"]],:y,:x,:soiltype)
habmat=Matrix(habmat[:,Not("y")])

#Threshold for a suitable habitat
mid=quantile(landscape[:,:soiltype],0.5)

#Define dispersal probability as a function of #neighbors for all three dispersal methods
comp_dist=10
maxnb=((2*comp_dist+1)^2)-1
neigh=log.(maxnb,1:maxnb)
comp_list=(fill(1,(length(neigh)+1)),[0.1; 0.1 .+ neigh],[1.0;1.0 .- neigh])  

d=Normal(0,disps/2)

result=DataFrame(rangepars=Int[],disp=Int[],method=Int[],x=Int[],y=Int[])


for p in 1:length(pars)
    
    disps=pars[p][1]
    methods=pars[p][2]

    for i in 1:100

        samp2=shuffle(findall(popmat.==1))[collect(1:100)]

        for j in 1:length(samp2)

            x=samp2[j][1]
            y=samp2[j][2]

            xs=((x-disps):(x+disps))
            xs=xs[0 .<xs .<=Ly]

            ys=(y-disps):(y+disps)
            ys=ys[0 .<ys .<=Lx]

            inds=collect(Iterators.product(xs,ys))

            vals=Float64[]            

            for k in 1:length(inds)

                xss=((inds[k][1]-comp_dist):(inds[k][1]+comp_dist))
                xss=xss[0 .<xss .<=Ly]

                yss=(inds[k][2]-comp_dist):(inds[k][2]+comp_dist) 
                yss=yss[0 .<yss .<=Lx]

                nb=sum(popmat[xss,yss])+1 

                val=pdf(d,sqrt(((x-inds[k][1])^2)+((y-inds[k][2])^2)))*                
                    comp_list[methods][nb]*
                    (sign(1-popmat[inds[k][1],inds[k][2]])) 

                push!(vals,val)

            end

            ind=wsample(collect(1:length(vals)),vals)

            popmat[inds[ind][1],inds[ind][2]]=1

        end

        #Apply habitat filtering to kill off trees in bad habitats
        samp=findall(popmat.==1)
        dths=length(samp)-abun
        mort=habmat[samp]
        del=findall(in.(mort,Ref(sort(mort)[1:dths])))
        popmat[samp[del]].=0

    end

    popmat1=DataFrame(popmat,:auto)
    rename!(x-> strip(string(x), ['x']),popmat1)
    popmat1[:,:y].=1:Ly
    popmat1=stack(popmat1,1:Lx)
    rename!(popmat1,:variable=> :x, :value => :pres)
    popmat1[!,:x] = parse.(Int64,popmat1[!,:x])
    filter!(:pres => x-> x .== 1,popmat1)

    append!(result,DataFrame(rangepars=rangepar,disp=disps,method=methods,x=popmat1.x,y=popmat1.y))  
end


In [ ]:
using  CSV, DataFrames, Random, LinearAlgebra, Distributions, Plots

Lx=1000 
Ly=500
abun=1000
timepoints=10000
nbirth=ndeath=convert(Int64,floor(abun*0.1))


cd("C:\\Users\\mihir\\Documents\\landscapes")

landscape=CSV.read(string("landscape_",1,".csv"), DataFrame)

#landscape.soiltype.=1

rangepar=landscape.rangepars[1]
      
landscape[:,:pres]=shuffle([ones(Int8,abun);zeros(Int8,(nrow(landscape)-abun))])

initial=filter(:pres => x-> x .== 1,landscape)

popmat=unstack(landscape[:,["y","x","pres"]],:y,:x,:pres)
popmat=Matrix(popmat[:,Not("y")])

habmat=unstack(landscape[:,["y","x","soiltype"]],:y,:x,:soiltype)
habmat=Matrix{Float64}(habmat[:,Not("y")])

#Threshold for a suitable habitat
mid=quantile(landscape[:,:soiltype],0.5)

#Define dispersal probability as a function of #neighbors for all three dispersal methods
comp_dist=10
maxnb=((2*comp_dist+1)^2)-1
neigh=log.(maxnb,1:maxnb)
comp_list=(fill(1,(length(neigh)+1)),[0.1; 0.1 .+ neigh],[1.0;1.0 .- neigh])  

d=Normal(0,disps/2)    

sum(popmat)

In [ ]:
disps=10

methods=2

for i in 1:100
    
        samp=findall(popmat.==1)
        mort=habmat[samp]
        del=wsample(1:length(mort),mort,ndeath)
        push!(ds,length(del))
        popmat[samp[del]].=0  
        
       samp2=shuffle(findall(popmat.==1))
    
        j=1
    
        while (sum(popmat)<(abun))

            x=samp2[j][1]
            y=samp2[j][2]

            xs=((x-disps):(x+disps))
            xs=xs[0 .<xs .<=Ly]

            ys=(y-disps):(y+disps)
            ys=ys[0 .<ys .<=Lx]

            inds=collect(Iterators.product(xs,ys))

            vals=Float64[]            

            for k in 1:length(inds)

                xss=((inds[k][1]-comp_dist):(inds[k][1]+comp_dist))
                xss=xss[0 .<xss .<=Ly]

                yss=(inds[k][2]-comp_dist):(inds[k][2]+comp_dist) 
                yss=yss[0 .<yss .<=Lx]

                nb=sum(popmat[xss,yss])+1 

                val=pdf(d,sqrt(((x-inds[k][1])^2)+((y-inds[k][2])^2)))*                
                    comp_list[methods][nb]*
                    (sign(1-popmat[inds[k][1],inds[k][2]])) 

                push!(vals,val)

            end

            ind=wsample(1:length(vals),vals)

            popmat[inds[ind][1],inds[ind][2]]=1
        
            j+=1

        end
        

    end

popmat1=DataFrame(popmat,:auto)
rename!(x-> strip(string(x), ['x']),popmat1)
popmat1[:,:y].=1:500
popmat1=stack(popmat1,1:1000)
rename!(popmat1,:variable=> :x, :value => :pres)
popmat1[!,:x] = parse.(Int64,popmat1[!,:x])
filter!(:pres => x-> x .== 1,popmat1)

In [ ]:
scatter(popmat1.x,popmat1.y)

In [ ]:
disps=10

methods=2

ds=[]
bs=[]

for i in 1:20

        samp2=shuffle(findall(popmat.==1))[collect(1:nbirth)]
    
        for j in 1:length(samp2)

            x=samp2[j][1]
            y=samp2[j][2]

            xs=((x-disps):(x+disps))
            xs=xs[0 .<xs .<=Ly]

            ys=(y-disps):(y+disps)
            ys=ys[0 .<ys .<=Lx]

            inds=collect(Iterators.product(xs,ys))

            vals=Float64[]            

            for k in 1:length(inds)

                xss=((inds[k][1]-comp_dist):(inds[k][1]+comp_dist))
                xss=xss[0 .<xss .<=Ly]

                yss=(inds[k][2]-comp_dist):(inds[k][2]+comp_dist) 
                yss=yss[0 .<yss .<=Lx]

                nb=sum(popmat[xss,yss])+1 

                val=pdf(d,sqrt(((x-inds[k][1])^2)+((y-inds[k][2])^2)))*                
                    comp_list[methods][nb]*
                    (sign(1-popmat[inds[k][1],inds[k][2]])) 

                push!(vals,val)

            end

            ind=wsample(1:length(vals),vals)

            popmat[inds[ind][1],inds[ind][2]]=1

        end
        
    
        samp=findall(popmat.==1)
        deaths=sum(popmat)-abun
        push!(ds,deaths)
        mort=habmat[samp]
        del=wsample(1:length(mort),mort,deaths)
        popmat[samp[del]].=0  
        
    end

popmat1=DataFrame(popmat,:auto)
rename!(x-> strip(string(x), ['x']),popmat1)
popmat1[:,:y].=1:500
popmat1=stack(popmat1,1:1000)
rename!(popmat1,:variable=> :x, :value => :pres)
popmat1[!,:x] = parse.(Int64,popmat1[!,:x])
filter!(:pres => x-> x .== 1,popmat1)


In [ ]:
disps=10

methods=2

ds=[]
bs=[]

for i in 1:20

        samp2=shuffle(findall(popmat.==1))
    
        j=1
    
        while (sum(popmat)<(abun+nbirth))

            x=samp2[j][1]
            y=samp2[j][2]

            xs=((x-disps):(x+disps))
            xs=xs[0 .<xs .<=Ly]

            ys=(y-disps):(y+disps)
            ys=ys[0 .<ys .<=Lx]

            inds=collect(Iterators.product(xs,ys))

            vals=Float64[]            

            for k in 1:length(inds)

                xss=((inds[k][1]-comp_dist):(inds[k][1]+comp_dist))
                xss=xss[0 .<xss .<=Ly]

                yss=(inds[k][2]-comp_dist):(inds[k][2]+comp_dist) 
                yss=yss[0 .<yss .<=Lx]

                nb=sum(popmat[xss,yss])+1 

                val=pdf(d,sqrt(((x-inds[k][1])^2)+((y-inds[k][2])^2)))*                
                    comp_list[methods][nb]*
                    (sign(1-popmat[inds[k][1],inds[k][2]])) 

                push!(vals,val)

            end

            ind=wsample(1:length(vals),vals)

            popmat[inds[ind][1],inds[ind][2]]=1
        
            j+=1

        end
        
    
        samp=findall(popmat.==1)
        deaths=sum(popmat)-abun
        push!(ds,deaths)
        mort=habmat[samp]
        del=wsample(1:length(mort),mort,deaths)
        push!(bs,del)
        popmat[samp[del]].=0  
        
        push!(bs,sum(popmat))

    end

popmat1=DataFrame(popmat,:auto)
rename!(x-> strip(string(x), ['x']),popmat1)
popmat1[:,:y].=1:500
popmat1=stack(popmat1,1:1000)
rename!(popmat1,:variable=> :x, :value => :pres)
popmat1[!,:x] = parse.(Int64,popmat1[!,:x])
filter!(:pres => x-> x .== 1,popmat1)

In [ ]:
scatter(popmat1.x,popmat1.y)

In [ ]:
ds

In [ ]:
using Distances, StatsPlots

In [ ]:
string("C:\\Users\\mihir\\Documents\\landscapes\\landscape_",1,".csv")

In [ ]:
data=CSV.read("C:\\Users\\mihir\\Documents\\landscapes\\final_1.csv",DataFrame)
dat= data[(data.rep .==1) .& (data.rangepar .==5) .& 
            (data.disp .==10) .& (data.method .=="positive"),:]

dat=transpose(Matrix(dat[:,[:x,:y]]))

adj=pairwise(Euclidean(),dat)
fg=FeaturedGraph(adj)
lp=normalized_laplacian(adj)
e=eigen(lp).values
e=e[e.>=1]



In [ ]:
landscape

In [ ]:
using  CSV, DataFrames, Random, LinearAlgebra, Distributions

Lx=1000 
Ly=500
abun=1000
timepoints=100
nbirth=ndeath=convert(Int64,floor(abun*0.1))
    
reps=collect(1:200)
disp=(2,5,10,15,30)
methodrefs=DataFrame(ind=[1, 2, 3], methods=["neutral","positive","negative"])
pars=collect(Iterators.product(disp,methodrefs[:,:ind]))

pars=pars[1:2]

reps=1

landscape=CSV.read(string("C:\\Users\\mihir\\Documents\\landscapes\\landscape_",reps,".csv"), DataFrame)

    repl=landscape[1,2]
    rangepar=landscape[1,3]    

    landscape[:,:pres]=shuffle([ones(Int8,abun);zeros(Int8,(nrow(landscape)-abun))])
    landscape[:,:rep]=fill(repl,nrow(landscape))
    landscape[:,:rangepars]=fill(rangepar,nrow(landscape))

    initial=filter(:pres => x-> x .== 1,landscape)
    initial=initial[:,[:rep,:rangepars,:x,:y,:pres]]
    
    CSV.write(string("C:\\Users\\mihir\\Documents\\landscapes2\\initial_",reps,".csv"),initial)

    init=unstack(landscape[:,["y","x","pres"]],:y,:x,:pres)
    init=Matrix(init[:,Not("y")])

    habmat=unstack(landscape[:,["y","x","soiltype"]],:y,:x,:soiltype)
    habmat=Matrix(habmat[:,Not("y")])
    
    #Threshold for a suitable habitat
    mid=quantile(landscape[:,:soiltype],0.5)
    
    result=DataFrame()
    
    i1=1
        
        disps=pars[i1][1]
        methods=pars[i1][2]
        
        #Define dispersal probability as a function of #neighbors for all three dispersal methods
        comp_dist=20
        maxnb=((2*comp_dist+1)^2)-1
        neigh=log.(maxnb,1:maxnb)
        comp_list=(fill(1,(length(neigh)+1)),[0.1; 0.1 .+ neigh],[1.0;1.0 .- neigh])    
        
        d=Normal(0,disps/2)
            
        popmat=init
            
        for i in 1:100
                
                #Apply habitat filtering to kill off trees in bad habitats
                samp=findall(popmat.==1)
                mort=habmat[samp]
                del=findall(in.(mort,Ref(sort(mort)[1:ndeath])))
                popmat[samp[del]].=0
                
                samp2=shuffle(findall(popmat.==1))[1:100]   

                for j in 1:length(samp2)

                    x=samp2[j][1]
                    y=samp2[j][2]

                    xs=((x-disps):(x+disps))
                    xs=xs[0 .<xs .<=Ly]

                    ys=(y-disps):(y+disps)
                    ys=ys[0 .<ys .<=Lx]

                    inds=collect(Iterators.product(xs,ys))

                    vals=Float64[]            

                    for k in 1:length(inds)

                        xss=((inds[k][1]-comp_dist):(inds[k][1]+comp_dist))
                        xss=xss[0 .<xss .<=Ly]

                        yss=(inds[k][2]-comp_dist):(inds[k][2]+comp_dist) 
                        yss=yss[0 .<yss .<=Lx]

                        nb=sum(popmat[xss,yss])+1 

                        val=pdf(d,sqrt(((x-inds[k][1])^2)+((y-inds[k][2])^2)))*                
                            comp_list[methods][nb]*
                            (sign(1-popmat[inds[k][1],inds[k][2]])) 

                        push!(vals,val)

                    end
                    
                    ind=wsample(collect(1:length(vals)),vals)

                    popmat[inds[ind][1],inds[ind][2]]=1
                end
        end
        
        
        popmat1=DataFrame(popmat,:auto)
        rename!(x-> strip(string(x), ['x']),popmat1)
        popmat1[:,:y].=1:Ly
        popmat1=stack(popmat1,1:Lx)
        rename!(popmat1,:variable=> :x, :value => :pres)
        popmat1[!,:x] = parse.(Int64,popmat1[!,:x])
        filter!(:pres => x-> x .== 1,popmat1)
        popmat1[:,:rep].=repl
        popmat1[:,:rangepar].=rangepar
        popmat1[:,:disp].=disps
        popmat1[:,:method].=methodrefs[methods,2]        
        append!(result,popmat1)
            



In [ ]:
combine(groupby(result, [:disp,:method]), nrow)

In [ ]:
init=unstack(landscape[:,["y","x","pres"]],:y,:x,:pres)
    init=Matrix(init[:,Not("y")])
sum(init)